In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('../data/processed/clean_kelowna_housing_data.csv')

In [8]:
df = df.drop(columns = {'Unnamed: 0', 'Unnamed: 0.1'})

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             1277 non-null   object 
 1   price            1277 non-null   int64  
 2   address          1277 non-null   object 
 3   Type             1277 non-null   object 
 4   nbeds            1277 non-null   int64  
 5   nbath            1277 non-null   int64  
 6   laundry          1277 non-null   object 
 7   heating          1277 non-null   object 
 8   parking          1277 non-null   object 
 9   air_conditioner  1277 non-null   object 
 10  furnished        1277 non-null   object 
 11  animal_friendly  1277 non-null   object 
 12  Area             1277 non-null   int64  
 13  lat              1277 non-null   float64
 14  lng              1277 non-null   float64
dtypes: float64(2), int64(4), object(9)
memory usage: 149.8+ KB


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,1277.0,1935.963195,1248.719288,400.00000,1000.00000,1800.00000,2500.00000,12785.00000
nbeds,1277.0,2.105717,1.221133,1.00000,1.00000,2.00000,3.00000,22.00000
nbath,1277.0,1.515270,0.741172,0.00000,1.00000,1.00000,2.00000,6.00000
Area,1277.0,1912.438528,17368.342577,2.00000,932.00000,1200.00000,1200.00000,440000.00000
lat,1277.0,49.887797,0.163663,49.29241,49.85989,49.88698,49.89780,50.38468
lng,1277.0,-119.472568,0.136139,-120.14867,-119.49691,-119.49661,-119.41109,-118.61706


In [16]:
df = df.drop(labels=[1206, 1014, 879, 854, 444], axis=0)

KeyError: '[1206, 1014, 879, 854, 444] not found in axis'

In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,1272.0,1939.996069,1249.481696,400.00000,1000.000000,1800.00000,2500.00000,12785.00000
nbeds,1272.0,2.110063,1.221558,1.00000,1.000000,2.00000,3.00000,22.00000
nbath,1272.0,1.516509,0.741940,0.00000,1.000000,1.00000,2.00000,6.00000
Area,1272.0,1219.419811,543.449017,111.00000,932.000000,1200.00000,1200.00000,7000.00000
lat,1272.0,49.887820,0.163433,49.29241,49.860230,49.88698,49.89791,50.38468
lng,1272.0,-119.472295,0.136056,-120.14867,-119.496653,-119.49661,-119.41109,-118.61706
